In [18]:
# Does numba work with annoy
import numpy as np
from annoy import AnnoyIndex
from numba import jit

@jit(fastmath=True, forceobj=True)
def test_numba():
    f = 40
    t = AnnoyIndex(f, 'euclidean')
    for i in range(10000):
        v = np.random.rand(f)
        t.add_item(i, v)
    t.build(10)
    for j in range(100000):
        t.get_nns_by_vector(np.random.rand(f), 10)

In [19]:
test_numba()

/tmp/ipykernel_996806/2664387536.py:6: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "test_numba" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_996806/2664387536.py (10)

File "../../../../../tmp/ipykernel_996806/2664387536.py", line 10:
<source missing, REPL/exec in use?>

  @jit(fastmath=True, forceobj=True)
/tmp/ipykernel_996806/2664387536.py:6: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "test_numba" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_996806/2664387536.py (13)

File "../../../../../tmp/ipykernel_996806/2664387536.py", line 13:
<source missing, REPL/exec in use?>

  @jit(fastmath=True, forceobj=True)


In [20]:
f = 40
t = AnnoyIndex(f, 'euclidean')
for i in range(10000):
    v = np.random.rand(f)
    t.add_item(i, v)
t.build(10)
for j in range(100000):
    t.get_nns_by_vector(np.random.rand(f), 10)

# Memory calculator

In [53]:

def memory_calculator(n_box, NPTS):
    """ Calculate the peak memory usage of this script for planning. """
    peak_mem_interp_in_bits = n_box**3 * 64 * 3 # complex 64
    peak_mem_interp_in_gigabytes = peak_mem_interp_in_bits / 8 / 1024 / 1024 / 1024
    peak_mem_fftw_in_bits = 2 * n_box**3 * 64 * 3
    peak_mem_fftw_in_gigabytes = peak_mem_fftw_in_bits / 8 / 1024 / 1024 / 1024
    return peak_mem_interp_in_gigabytes, peak_mem_fftw_in_gigabytes

In [55]:
memory_calculator(500, 500)

(2.7939677238464355, 5.587935447692871)

In [5]:
import numpy as np
np.float16(1/3)

0.3333

In [6]:
import math
def planner(n_total_res, l_total_length, n_box_affordable, n_total_threads):
    m = math.cbrt(n_total_threads)  # number of threads in each dimension = the folding factor
    assert m.is_integer()==True, 'Number of threads must be a cube of an integer. Support for any number is not yet implemented.'
    m = int(m)

    n_loops_per_axis = 1
    n_full_box = n_total_res / m
    assert n_full_box.is_integer()==True, 'Divided Nbox must be an integer.'

    Nbox = n_full_box
    while Nbox > n_box_affordable or Nbox.is_integer()==False:
        n_loops_per_axis += 1
        Nbox = n_full_box / n_loops_per_axis
    n_loops = n_loops_per_axis**3
    Nbox = int(Nbox) # number of pixels in each dimension of the box

    Lbox = Nbox / n_total_res * l_total_length # length of the box

    return n_loops, m, Nbox, Lbox

In [17]:
planner(10000, 1, 800, 1000)

(8, 10, 500, 0.05)

# How does reshape work?

In [15]:
import numpy as np
np.random.seed(0)
a = np.empty((8, 8, 100, 3))
for thread in range(8):
    for loop in range(8):
        for qidx in range(100):
            for vec in range(3):
                a[thread, loop, qidx, vec] = np.random.rand()
print(a.shape, a[0,1,:])

(8, 8, 100, 3) [[0.9065555  0.77404733 0.33314515]
 [0.08110139 0.40724117 0.23223414]
 [0.13248763 0.05342718 0.72559436]
 [0.01142746 0.77058075 0.14694665]
 [0.07952208 0.08960303 0.67204781]
 [0.24536721 0.42053947 0.55736879]
 [0.86055117 0.72704426 0.27032791]
 [0.1314828  0.05537432 0.30159863]
 [0.26211815 0.45614057 0.68328134]
 [0.69562545 0.28351885 0.37992696]
 [0.18115096 0.78854551 0.05684808]
 [0.69699724 0.7786954  0.77740756]
 [0.25942256 0.37381314 0.58759964]
 [0.2728219  0.3708528  0.19705428]
 [0.45985588 0.0446123  0.79979588]
 [0.07695645 0.51883515 0.3068101 ]
 [0.57754295 0.95943334 0.64557024]
 [0.03536244 0.43040244 0.51001685]
 [0.53617749 0.68139251 0.2775961 ]
 [0.12886057 0.39267568 0.95640572]
 [0.18713089 0.90398395 0.54380595]
 [0.45691142 0.88204141 0.45860396]
 [0.72416764 0.39902532 0.90404439]
 [0.69002502 0.69962205 0.3277204 ]
 [0.75677864 0.63606106 0.24002027]
 [0.16053882 0.79639147 0.9591666 ]
 [0.45813883 0.59098417 0.85772264]
 [0.45722345 

In [16]:
b = a.reshape(64, 100, 3)
b[1,:]

array([[0.9065555 , 0.77404733, 0.33314515],
       [0.08110139, 0.40724117, 0.23223414],
       [0.13248763, 0.05342718, 0.72559436],
       [0.01142746, 0.77058075, 0.14694665],
       [0.07952208, 0.08960303, 0.67204781],
       [0.24536721, 0.42053947, 0.55736879],
       [0.86055117, 0.72704426, 0.27032791],
       [0.1314828 , 0.05537432, 0.30159863],
       [0.26211815, 0.45614057, 0.68328134],
       [0.69562545, 0.28351885, 0.37992696],
       [0.18115096, 0.78854551, 0.05684808],
       [0.69699724, 0.7786954 , 0.77740756],
       [0.25942256, 0.37381314, 0.58759964],
       [0.2728219 , 0.3708528 , 0.19705428],
       [0.45985588, 0.0446123 , 0.79979588],
       [0.07695645, 0.51883515, 0.3068101 ],
       [0.57754295, 0.95943334, 0.64557024],
       [0.03536244, 0.43040244, 0.51001685],
       [0.53617749, 0.68139251, 0.2775961 ],
       [0.12886057, 0.39267568, 0.95640572],
       [0.18713089, 0.90398395, 0.54380595],
       [0.45691142, 0.88204141, 0.45860396],
       [0.

# rst assignment

In [6]:
n_threads_per_axis = 2
n = 2
for rank in range(8):
    r = n * ((rank % n_threads_per_axis) // 1)
    s = n * ((rank % n_threads_per_axis**2) // n_threads_per_axis)
    t = n * (rank // n_threads_per_axis**2)
    print(r,s,t)

0 0 0
2 0 0
0 2 0
2 2 0
0 0 2
2 0 2
0 2 2
2 2 2
